# Model Selection 모듈
붗꽃 데이터 세트 사용

## 1. train_test_split()
 학습용 데이터와 테스트용 데이터를 분리  
 
 <span style="font-size:18px"><span style="font-weight:bold">X_train, X_test, y_train, y_test</span> = train_test_split(<span style="font-weight:bold">x_data set(feature), y_data set(label), test_size, random_state</span>)</span>
 + test_size : 전체 데이터에서 테스트 데이터 크기의 비율(ex. test_size=0.2이면 테스트데이터 20%, 학습 데이터 80%)  
 + random_state : 호출할 때마다 같은 학습/데이터 데이터 세트를 생성하기 위한 난수 발생 값
 <br><br>
 + <span style="font-weight:bold">X_train</span> : 학습용 데이터의 피처 데이터 세트  
 + <span style="font-weight:bold">X_test</span> : 테스트용 데이터의 피처 데이터 세트  
 + <span style="font-weight:bold">y_train</span> : 학습용 데이터의 레이블 데이터 세트  
 + <span style="font-weight:bold">y_test</span> : 테스트용 데이터의 레이블 데이터 세트  

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier()
iris = load_iris()

# 학습데이터와 테스트데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=121)

#모델 학습 및 예측
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도 : {0:.4f}' .format(accuracy_score(y_test, pred)))

예측 정확도 : 0.9556


## 2. 교차검증
 + 별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습 및 평가를 수행하는 것  
 + 과적합(overfitting) 문제 해결
 
### K 폴드 교차 검증
  + 가장 보편적으로 사용되는 교차 검증 기법
  + K 개의 데이터 폴드 세트를 만든 후, K번만큼 각 폴드 세트에 학습, 검증평가를 반복적으로 수행하는 방법
  + 사이킷런에서 <span style="font-weight:bold">KFold</span> 와 <span style="font-weight:bold">StratifiedKFold</span> 클래스 제공

### KFold 교차 검증
---

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체 생성
kfold = KFold(n_splits=5)

# 폴드 세트별로 정확도를 담을 리스트 생성
cv_accuracy = []

#전체 데이터가 150개이므로, 학습용 데이터 세트는 120개(4/5), 검증 테스트 데이터 세트는 30개(1/5)
#반복 횟수 측정를 측정하는 n_iteration
n_iteration = 0

# split()으로 폴드 별 학습용, 검증용 테스트의 인덱스를 array로 반환
for train_index, test_index, in kfold.split(features):
    # 학습용, 검증용 테스트의 피처 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    # 학습용, 검증용 테스트의 레이블 데이터 추출
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iteration+=1
    
    #정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    
    # 각 폴드의 데이터 세트가 잘 분할됐는지 출력
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print("\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}" .format(n_iteration, accuracy, train_size, test_size))
    print("#{0} 검증 세트 인덱스 : {1}" .format(n_iteration, test_index))
    
    #리스트에 각 폴드별 정확도 추가
    cv_accuracy.append(accuracy)
    

#5개 폴드의 평균 정확도 계산
print("\n\n## 평균 검증 정확도 : ", np.mean(cv_accuracy))


#1 교차 검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30
#5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]


## 평균 검증 정확도 :  0.9


### Stratified KFold 교차 검증
---
+ 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식
+ 불균형한 분포도의 레이블 데이터 경우 학습/테스트 데이터를 제대로 분배하지 못함
    1. 특정 레이블 값이 지나치게 많을 경우
    2. 특정 레이블 값이 지나치게 적을 경우  
    
<span style="font-weight:bold">--> Stratified KFold 로 해결</span>

---
#### 불균형한 분포도를 가진 레이블 데이터 세트를 분할할 경우
---
--> 학습 데이터 레이블과 검증 데이터 레이블이 완전 다른 값으로 추출되어 예측 불가

In [3]:
#붓꽃 데이터를 임의로 불균형한 분포도를 가지도록 생성함
import pandas as pd
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target

kfold = KFold(n_splits=3)
n_iteration = 0

for train_index, test_index, in kfold.split(iris_df):
    n_iteration+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print("## 교차검증 : {0}" .format(n_iteration))
    print("학습 레이블 데이터 분포 :\n", label_train.value_counts())
    print("검증 레이블 데이터 분포 :\n", label_test.value_counts())
    print("\n\n")

## 교차검증 : 1
학습 레이블 데이터 분포 :
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 0    50
Name: label, dtype: int64



## 교차검증 : 2
학습 레이블 데이터 분포 :
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    50
Name: label, dtype: int64



## 교차검증 : 3
학습 레이블 데이터 분포 :
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 2    50
Name: label, dtype: int64





---
####  StratifiedKFold 로 데이터 세트를 분할할 경우
---
--> 학습 데이터 레이블과 검증 데이터 레이블 값의 분포도가 동일하게 분할되었음  
```
for train_index, test_index, in skf.split(iris_df, iris_df['label']):
```
split() 메서드에 인자로 <span style="font-weight:bold">레이블 데이터 세트 (iris_df['lable'])</span> 도 필요함

In [4]:
from sklearn.model_selection import StratifiedKFold

# StratifiedKFold 객체 생성
skf = StratifiedKFold(n_splits=3)
n_iteration=0

for train_index, test_index, in skf.split(iris_df, iris_df['label']):
    n_iteration+=1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    
    print("## 교차검증 : {0}" .format(n_iteration))
    print("학습 레이블 데이터 분포 :\n", label_train.value_counts())
    print("검증 레이블 데이터 분포 :\n", label_test.value_counts())
    print("\n\n")

## 교차검증 : 1
학습 레이블 데이터 분포 :
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 0    17
1    17
2    16
Name: label, dtype: int64



## 교차검증 : 2
학습 레이블 데이터 분포 :
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 0    17
2    17
1    16
Name: label, dtype: int64



## 교차검증 : 3
학습 레이블 데이터 분포 :
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포 :
 1    17
2    17
0    16
Name: label, dtype: int64





In [5]:
dt_clf = DecisionTreeClassifier(random_state=156)

# 3개의 폴드 세트로 분리하는 StratifiedKFold 객체 생성
skfold = StratifiedKFold(n_splits=3)

# 폴드 세트별로 정확도를 담을 리스트 생성
cv_accuracy = []

#전체 데이터가 150개이므로, 학습용 데이터 세트는 120개(4/5), 검증 테스트 데이터 세트는 30개(1/5)
#반복 횟수 측정를 측정하는 n_iteration
n_iteration = 0


# split()으로 폴드 별 학습용, 검증용 테스트의 인덱스를 array로 반환
# 이 때, 레이블 데이터 세트도 반드시 추가 입력해야함
for train_index, test_index, in skfold.split(features, label):
    # 학습용, 검증용 테스트의 피처 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    # 학습용, 검증용 테스트의 레이블 데이터 추출
    y_train, y_test = label[train_index], label[test_index]
    
    #학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iteration+=1
    
    #정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    
    # 각 폴드의 데이터 세트가 잘 분할됐는지 출력
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print("\n#{0} 교차 검증 정확도 : {1}, 학습 데이터 크기 : {2}, 검증 데이터 크기 : {3}" .format(n_iteration, accuracy, train_size, test_size))
    print("#{0} 검증 세트 인덱스 : {1}" .format(n_iteration, test_index))
    
    #리스트에 각 폴드별 정확도 추가
    cv_accuracy.append(accuracy)
    

#교차 검증별 정확도 및 평균 정확도 계산
print("\n\n## 교차 검증별 정확도 : ", np.round(cv_accuracy, 4))
print("## 평균 검증 정확도 : ", np.mean(cv_accuracy))


#1 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도 : 0.94, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#2 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50
#3 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]


## 교차 검증별 정확도 :  [0.98 0.94 0.98]
## 평균 검증 정확도 :  0.9666666666666667


## corss_val_score()
---
 사이킷런에서 제공하는 교차 검증을 편리하게 수행할 수 있는 API  

<span style="font-size:18px">corss_val_score(<span style="font-weight:bold">estimator, X, y, scoring, cv</span>)</span>
+ <span style="font-weight:bold">estimator</span> : 사이킷런의 Classifier(분류 알고리즘 클래스) or Regressor(회귀 알고리즘 클래스)
+ <span style="font-weight:bold">X</span> : 피처 데이터 세트
+ <span style="font-weight:bold">y</span> : 레이블 데이터 세트
+ <span style="font-weight:bold">scoring</span> : 예측 성능 평가 지표.  scoring으로 지정된 성능 지표 측정값을 배열 형태로 반환
+ <span style="font-weight:bold">cv</span> : 교차 검증 폴드 수

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score, cross_validate

iris = load_iris()

dt_clf = DecisionTreeClassifier(random_state=156)
data = iris.data
label = iris.target

# 성능 지표(scoring)는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print("교차 검증별 정확도", np.round(scores, 4))
print("평균 검증 정확도", np.round(np.mean(scores), 4))

교차 검증별 정확도 [0.98 0.94 0.98]
평균 검증 정확도 0.9667


## GridSearchCV
---
 교차 검증을 기반으로 하이퍼 파라미터의 최적 값을 찾을 수 있는 방식  
 몇 개의 하이퍼 파라미터를 입력하고, 교차 검증을 통해서 최적의 하이퍼 파라미터를 찾는다.
 <br><br>
 <span style="font-size:18px">GridSearchCV(<span style="font-weight:bold">estimator, param_grid, scoring, cv, refit</span>)</span>
 + <span style="font-weight:bold">estimator</span> : Classifier, Regressor, Pipeline 사용
 + <span style="font-weight:bold">param_grid</span> : key + 리스트 값을 가지는 딕셔너리. estimator 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정함
 + <span style="font-weight:bold">scoring</span> : 예측 성능 평가 지표
 + <span style="font-weight:bold">cv</span> : 교차 검증 폴드 수
 + <span style="font-weight:bold">refit</span> : 디폴트 True. Ture일 때, 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킴
 

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
import pandas as pd

#  데이터 로딩 및 학습/테스트데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

## 파라미터를 딕셔너리 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

## GridSearchCV 객체 생성
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 붓꽃 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습 및 평가
grid_dtree.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [8]:
#최적의 하이퍼 파라미터 값과 정확도 추출
print('GridSearchCV 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}' .format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750


In [9]:
# GridSearchCV 의 refit으로 이미 학습된 estimator 반환 및 예측, 성능평가

# GridSearchCV 의 refit으로 학습된 estimator 반환
estimator = grid_dtree.best_estimator_

# estimator은 이미 학습이 된 상태이므로 fit 할 필요가 없음
# 예측 및 성능평가
pred = estimator.predict(X_test)
print('GridSearchCV를 사용한 테스트 데이터 세트 정확도 : {0:.4f}' .format(accuracy_score(y_test, pred)))

GridSearchCV를 사용한 테스트 데이터 세트 정확도 : 0.9667
